In [2]:
%load_ext autoreload
%autoreload 2


# Traci

In [3]:
path='/usr/share/sumo/tests/complex/tutorial/traci_tls'
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import random

# we need to import python modules from the $SUMO_HOME/tools directory
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from sumolib import checkBinary  # noqa
import traci  # noqa

In [4]:
# this script has been called from the command line. It will start sumo as a
# server, then connect and run
sumoBinary = checkBinary('sumo')


# this is the normal way of using traci. sumo is started as a
# subprocess and then the python script connects and runs
traci.start([sumoBinary, "-c", f"{path}/data/cross.sumocfg",
                         "--tripinfo-output", "tripinfo.xml"])

 Retrying in 1 seconds


/Users/liuyuchen/opt/anaconda3/envs/atsc/lib/python3.8/site-packages/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 62485 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


/Users/liuyuchen/opt/anaconda3/envs/atsc/lib/python3.8/site-packages/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 62488 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


...

/Users/liuyuchen/opt/anaconda3/envs/atsc/lib/python3.8/site-packages/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 62503 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


/Users/liuyuchen/opt/anaconda3/envs/atsc/lib/python3.8/site-packages/traci/main.py:174: UserWarning: Could not connect to TraCI server using port 62506 (TraCI server already finished). Retrying with different port.
  warnings.warn(("Could not connect to TraCI server using port %s (%s)." +


 Retrying in 1 seconds


KeyboardInterrupt: 

## Run a simulation until all vehicles have arrived

In [ ]:
while traci.simulation.getMinExpectedNumber() > 0:
  traci.simulationStep()
traci.close()

## TrafficLights

In [ ]:
traci.simulationStep()
if traci.trafficlight.getPhase("0") == 2:
    # we are not already switching
    if traci.inductionloop.getLastStepVehicleNumber("0") > 0:
        # there is a vehicle from the north, switch
        traci.trafficlight.setPhase("0", 3)
    else:
        # otherwise try to keep green for EW
        traci.trafficlight.setPhase("0", 2)

## Vehicles

In [ ]:
while traci.simulation.getMinExpectedNumber() > 0:
    traci.simulationStep()
    if len(traci.vehicle.getIDList()) > 0:
        break

In [ ]:
traci.vehicle.getIDList()

In [ ]:
traci.vehicle.getSpeed("left_0")
traci.vehicle.getPosition("left_0")

# ATSC

## Large Grid
25 agents, 12 dim observation

In [ ]:
from algorithms.envs.NCS.large_grid_env import LargeGridEnv
import os
import configparser
import traci

In [ ]:
config_path = "algorithms/envs/NCS/config/config_ma2c_nc_grid.ini"
config_path = os.path.join(os.path.dirname("."), config_path)
config = configparser.ConfigParser()
config.read(config_path)
config = config['ENV_CONFIG']
env = LargeGridEnv(config)

In [ ]:
state = env.reset()

In [ ]:
s, r, d, i = env.step([0]*25)

## Real Net
28 agents, observation space from 2 dim to 22 dim, action space 2 to 6.

In [3]:
from algorithms.envs.NCS.real_net_env import RealNetEnv
import os
import configparser
import traci
import numpy as np

config_path = "algorithms/envs/NCS/config/config_ma2c_nc_net.ini"
config_path = os.path.join(os.path.dirname("."), config_path)
config = configparser.ConfigParser()
config.read(config_path)
config = config['ENV_CONFIG']
env = RealNetEnv(config)
env.terminate()

Initializing sumo with label 472907
 Retrying in 1 seconds


TraCIException: Connection '472907' is not known.

In [ ]:
state = env.reset()

In [ ]:
env.node_names
env.phase_node_map
env.phase_map.phases

In [ ]:
phases = [env.phase_node_map[node] for node in env.node_names]
n_action = [env.phase_map.get_phase_num(item) for item in phases]
state, reward, done, info = env.step(np.array(n_action)-1)

## Why is not it symetric?

In [63]:
env.neighbor_map
np.zeros()

{'10026': ['9431', '9561', 'cluster_9563_9597', '9531'],
 '8794': ['cluster_8985_9609', '9837', '9058', 'cluster_9563_9597'],
 '8940': ['9007', '9429'],
 '8996': [],
 '9007': ['9309', '8940'],
 '9058': ['cluster_8985_9609', '8794', 'joinedS_0'],
 '9153': ['9643'],
 '9309': ['9466', '9007', 'cluster_9043_9052'],
 '9413': ['9721', '9837'],
 '9429': ['cluster_9043_9052', '8940'],
 '9431': ['9721', '9884', '9561', '10026'],
 '9433': [],
 '9466': ['9309', 'joinedS_0'],
 '9480': [],
 '9531': ['joinedS_1'],
 '9561': ['cluster_9389_9689', '10026'],
 '9643': ['9153'],
 '9713': ['9721'],
 '9721': ['9431', '9713', '9413'],
 '9837': ['9413', '8794', 'cluster_8985_9609'],
 '9884': ['9713', 'cluster_9389_9689'],
 'cluster_8751_9630': [],
 'cluster_8985_9609': ['9837', '8794', '9058'],
 'cluster_9043_9052': ['cluster_9563_9597', '10026', 'joinedS_1'],
 'cluster_9389_9689': ['cluster_8751_9630', '9884', '9561', '8996'],
 'cluster_9563_9597': ['10026', '8794', 'joinedS_0', 'cluster_9043_9052'],
 'joine

In [62]:
radius = 1
x = env.neighbor_mask
x = x + x.transpose(1, 0) + np.eye(*x.shape, dtype="int")
result = np.eye(*x.shape, dtype='int')
for i in range(radius):
    result = result.dot(x)
result = result.clip(0, 1)
print(f"neighbors: {result.sum(axis=1)}")

neighbors: [6 5 3 2 3 4 2 4 3 4 5 1 3 1 3 4 2 3 4 4 4 2 4 6 5 5 4 4]
